In [ ]:
# add project paths 
import os
import sys

# get current directory and project root
current_dir = os.getcwd()  # Fixed: was getcdw()
project_root = os.path.abspath(os.path.join(current_dir,".."))

# add all necessary paths
paths_to_add = [
    os.path.join(project_root, "src","init"),
    project_root,
    os.path.join(project_root, "config"),
    os.path.join(project_root, "src","agent"),
    os.path.join(project_root, "src","agent","tools"),
    os.path.join(project_root, "src"),
    os.path.join(project_root, "deployment"),
]

for path in paths_to_add:
    if path not in sys.path:  # Fixed: was sys.paths
        sys.path.append(path)

In [ ]:
# install dependencies

import subprocess
import sys
import os

requirements_file = os.path.join(project_root,"requirements.txt")
subprocess.check_all([sys.executable,"-m","-pip","install","-r",requirements_file])

In [ ]:
# core imports

import mlflow
import mlflow.pyfunc
 
# import modules ... 


In [ ]:
# model registration 
from mlflow.models.resources import DatabricksServingEndpoint

catalog_name = "catalog_name"
schema_name = "schema_name"
agent_name = "agent_name"
model_name = f"{catalog_name}.{schema_name}.{agent_name}"
model_version = "1.0.0"
wheel_name = "xxx.whl"
wheel_path = os.path.join(project_root,"deployment",wheel_name)

def register_model_to_mlflow():
  try:
      if mlflow.active_run() is not None:
          mlflow.end_run()
      
      mlflow.set_registry_uri("databricks-uc")

      with mlflow.start_run(run_name = f"register_{model_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}") as run:
          
          model_info = mlflow.pyfunc.log_model(
              python_model = "../copilot_name_placeholder.py",
              artifact_path = "agent",
              registered_model_name = model_name,
              code_paths = [wheel_path],
              pip_requirements = [f"./code/{wheel_name}"],
              resources = [DatabricksServingEndpoint(endpoint_name="databricks-claude-sonnet-4")]
          )

  except Exception as e:
        print(f"Model registration failed: {str(e)}")
        return {'status':'error','error':str(e)}

registration_result = register_model_to_mlflow()

In [ ]:
# deploy the model as a serving endpoint
from mlflow.tracking import MLflowClient
from databricks import agents

def get_latest_model_version(model_name):
  mlflow_client = MLflowClient(registry_uri="databricks-uc")
  latest_version = 1
  for mv in mlflow_client.search_model_versions(f"name='{model_name}'"):
      version_int = int(mv.version)
      if version_int > latest_version:
         latest_version = version_int
  return latest_version

endpoint_name = 'placeholder'

agents.deploy(
    model_name=model_name,
    model_version=get_latest_model_version(model_name),
     endpoint_name = endpoint_name,
     scale_to_zero_enabled = True
     )